In [ ]:
pip install transformers datasets tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# Sample data
texts = [
"I love this movie!",
"This is terrible.",
"I feel amazing today.",
"Worst experience ever.",
"Best product I bought!",
"I hate this."
]
labels = [1, 0, 1, 0, 1, 0] # 1 = Positive, 0 = Negative

In [ ]:
# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(
texts, labels, test_size=0.2, random_state=42
)

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name, local_files_only=False)

# Tokenize
train_encodings = tokenizer(train_texts, truncation=True,padding=True, return_tensors="tf")
val_encodings = tokenizer(val_texts, truncation=True,padding=True, return_tensors="tf")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
from transformers import TFAutoModel

# Load base BERT model (without the classification head)
bert_base = TFAutoModel.from_pretrained(model_name)

# Build the fine-tuning model
input_ids = tf.keras.Input(shape=(None,), dtype=tf.int32,name="input_ids")
attention_mask = tf.keras.Input(shape=(None,), dtype=tf.int32,name="attention_mask")

outputs = bert_base(input_ids, attention_mask=attention_mask)
cls_output = outputs.last_hidden_state[:, 0, :] # Take the [CLS]token

x = tf.keras.layers.Dense(128, activation='relu')(cls_output)
x = tf.keras.layers.Dropout(0.2)(x)
final_output = tf.keras.layers.Dense(1, activation='sigmoid')(x) #Binary classification

model = tf.keras.Model(inputs=[input_ids, attention_mask],
outputs=final_output)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5),
loss='binary_crossentropy',
metrics=['accuracy'])

model.fit(
x={'input_ids': train_encodings['input_ids'], 'attention_mask':
train_encodings['attention_mask']},
y=np.array(train_labels),
validation_data=(
{'input_ids': val_encodings['input_ids'], 'attention_mask':
val_encodings['attention_mask']},

np.array(val_labels)
),batch_size=2,epochs=3)

Epoch 1/3


2/2 [==============================] - 37s 4s/step - loss: 0.2982 - accuracy: 1.0000 - val_loss: 0.3064 - val_accuracy: 1.0000
Epoch 2/3
2/2 [==============================] - 4s 2s/step - loss: 0.1314 - accuracy: 1.0000 - val_loss: 0.2320 - val_accuracy: 1.0000
Epoch 3/3
2/2 [==============================] - 3s 2s/step - loss: 0.0441 - accuracy: 1.0000 - val_loss: 0.1827 - val_accuracy: 1.0000


In [ ]:
def predict_sentiment(text):
  tokens = tokenizer(text, return_tensors="tf", truncation=True,padding=True)
  prediction = model.predict({'input_ids': tokens['input_ids'],'attention_mask': tokens['attention_mask']})
  return "Positive" if prediction[0][0] > 0.5 else "Negative"

print(predict_sentiment("I absolutely loved it!"))
print(predict_sentiment("This is the worst thing ever."))


1/1 [==============================] - 5s 5s/step
Positive
1/1 [==============================] - 10s 10s/step
Negative


In [ ]:
accuracy = model.evaluate(
    x={'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask']},
    y=np.array(val_labels),
    verbose=0 )

print(f"Accuracy: {accuracy[1]:.2f}")

Accuracy: 1.00
